# Chapter 8: Panel Data

https://mixtape.scunning.com/08-panel_data

## Intro

Panel data estimators (models) are among the most important tools in the causal inference toolkit. The estimators are designed explicitly for longitudinal data—the repeated observing of a unit over time. Under certain situations, repeatedly observing the same unit over time can overcome a particular kind of omitted variable bias, though not all kinds. While it is possible that observing the same unit over time will not resolve the bias, there are still many applications where it can, and that’s why this method is so important

## DAG Example

Before I dig into the technical assumptions and estimation methodology for panel data techniques, I want to review a simple DAG illustrating those assumptions. This DAG comes from [Imai and Kim (2017)](https://mixtape.scunning.com/references#ref-Imai2017). 

Let’s say that we have data on a column of outcomes $Y_i$, which appear in three time periods (here notated as $Y_{i1}$, $Y_{i2}$ and $Y_{i3}$). In our notation $t = 1,2,3$ indexes the time period where each $i$ unit is observed. Likewise, we have a matrix of covariates $D_i$. These also vary over time (noted as $D_{i1}$, $D_{i2}$ and $D_{i3}$). 

Also, there exists a single unit-specific unobserved variable $u_i$, which varies across units, but which does not vary over time for that unit. Hence the reason that there is no $t = 1,2,3$ subscript for our $u_i$ variable. Key to this variable is (a) it is unobserved in the data set, (b) it is unit-specific, and (c) it does not change over time for a given unit . 

Finally there exists some unit-specific time-invariant variable, $X_i$ . Notice that it doesn’t change over time, just like $u_i$, but unlike $u_i$ it **is** observed.

<img src="img/panel_dag.png" alt="DAG" style="width: 350px;"/>

- First, let us note that $D_{i1}$ causes both $Y_{i1}$ as well as the next period’s treatment value, $D_{i2}$. 

- Second, note that an unobserved confounder, $u_i$, determines all $Y$ and all $D$ variables. Consequently, $D$ is **endogenous** since $u_i$ is unobserved and absorbed into the structural error term of the regression model. 

- Thirdly, there is no time-varying unobserved confounder correlated with $D_{it}$—the only confounder is $u_i$, which we call the **unobserved heterogeneity**. 

- Fourth, past outcomes do not directly affect current outcomes (i.e., no direct edge between the $Y_{it}$ variables). 

- Fifth, past outcomes do not directly affect current treatments (i.e., no direct edge from $Y_{i,t-1}$ to $D_{it}$). 

- And finally, past treatments, $D_{i,t-1}$ do not directly affect current outcomes, $Y_{it}$ (i.e., no direct edge from  $Y_{i,t-1}$ and $D_{it}$).

It is under these assumptions that we can use a particular panel method called fixed effects to isolate the causal effect of $D$ on $Y$.

What might an example of this be? Let’s return to our story about the returns to education. Let’s say that we are interested in the effect of schooling on earnings, and schooling is partly determined by unchanging genetic factors which themselves determine unobserved ability, like intelligence, contentiousness, and motivation (Conley and Fletcher 2017). If we observe the same people’s time-varying earnings and schoolings over time, then if the situation described by the above DAG describes both the directed edges and the missing edges, then we can use panel fixed effects models to identify the causal effect of schooling on earnings.

## Estimation
When we use the term “panel data,” what do we mean? We mean a data set where we observe the same units (e.g., individuals, firms, countries, schools) over more than one time period. Often our outcome variable depends on several factors, some of which are observed and some of which are unobserved in our data, and insofar as the unobserved variables are correlated with the treatment variable, then the treatment variable is endogenous and correlations are not estimates of a causal effect. This chapter focuses on the conditions under which a correlation between  and  reflects a causal effect even with unobserved variables that are correlated with the treatment variable. Specifically, if these omitted variables are constant over time, then even if they are heterogeneous across units, we can use panel data estimators to consistently estimate the effect of our treatment variable on outcomes.

There are several different kinds of estimators for panel data, but we will in this chapter only cover two: pooled ordinary least squares (POLS) and fixed effects (FE).

## Pooled OLS

The first estimator we will discuss is the pooled ordinary least squares, or POLS estimator. When we ignore the panel structure and regress $Y_{it}$ on $D_{it}$ we get:

$$
Y_{it} = \delta D_{it} + \eta_{it}; t = 1,2,...,T
$$

with composite error:

$$
\eta_{it} = c_i + \epsilon_{it}
$$

While our DAG did not include $\epsilon_{it}$, this would be equivalent to assuming that the unobserved heterogeneity, $c_i$, was uncorrelated with $D_{it}$ for all time periods.

**But this is not an appropriate assumption in our case because our DAG explicitly links the unobserved heterogeneity to both the outcome and the treatment in each period**. Or using our schooling-earnings example, schooling is likely based on unobserved background factors, $u_i$, and therefore without controlling for it, we have omitted variable bias and $\hat{\delta}$ is biased. No correlation between $D_{it}$ and $\eta_{it}$ necessarily means no correlation between the unobserved $u_i$ and $D_{it}$ for all $t$ and that is just probably not a credible assumption. An additional problem is that $\eta_{it}$ is serially correlated for unit $i$ since $u_i$ is present in each $t$ period. And thus heteroskedastic robust standard errors are also likely too small.

## Fixed effects (within Estimator)

Let’s rewrite our unobserved effects model so that this is still firmly in our minds:

$$
Y_{it} = \delta D_{it} + u_i + \epsilon_{it}; t = 1,2,...,T
$$

If we have data on multiple time periods, we can think of $u_i$ as fixed effects to be estimated. OLS estimation with fixed effects yields:

$$
\big(\widehat{\delta}, \widehat{u}_1, \dots, \widehat{u}_N\big) = \underset{b,m_1,\dots,m_N}{\arg\min} \sum_{i=1}^N\sum_{t=1}^T (Y_{it}-D_{it}b- m_i)^2
$$

<p>In case it isn’t clear, though, running a regression with the time-demeaned variables <span class="math inline">\(\ddot{Y}_{it}\equiv Y_{it} - \overline{Y}_i\)</span> and <span class="math inline">\(\ddot{D}_{it} \equiv D_{it} - \overline{D}\)</span> is <em>numerically equivalent</em> to a regression of <span class="math inline">\(Y_{it}\)</span> on <span class="math inline">\(D_{it}\)</span> and unit-specific dummy variables. Hence the reason this is sometimes called the “within” estimator, and sometimes called the “fixed effects” estimator. And when year fixed effects are included, the “twoway fixed effects” estimator. They are the same thing.</p>

Where’d the unobserved heterogeneity go?! It was deleted when we time-demeaned the data. And as we said, including individual fixed effects does this time demeaning automatically so that you don’t have to go to the actual trouble of doing it yourself manually

## Data Exercise: Survey of Adult Service Providers

Next I’d like to introduce a Stata exercise based on data collection for my own research: a survey of sex workers. You may or may not know this, but the Internet has had a profound effect on sex markets. It has moved sex work indoors while simultaneously breaking the traditional link between sex workers and pimps. It has increased safety and anonymity, too, which has had the effect of causing new entrants. The marginal sex worker has more education and better outside options than traditional US sex workers (Cunningham and Kendall 2011, 2014, 2016). The Internet, in sum, caused the marginal sex worker to shift towards women more sensitive to detection, harm, and arrest.

In 2008 and 2009, I surveyed (with Todd Kendall) approximately 700 US Internet-mediated sex workers. The survey was a basic labor-market survey; I asked them about their illicit and legal labor-market experiences, and about demographics. The survey had two parts: a “static” provider-specific section and a “panel” section. The panel section asked respondents to share information about each of the previous four sessions with clients.

I have created a shortened version of the data set and uploaded it to Github. It includes a few time-invariant provider characteristics, such as race, age, marital status, years of schooling, and body mass index, as well as several time-variant session-specific characteristics including the log of the hourly price, the log of the session length (in hours), characteristics of the client himself, whether a condom was used in any capacity during the session, whether the client was a “regular,” etc.

In this exercise, you will estimate three types of models: a pooled OLS model, a fixed effects (FE), and a demeaned OLS model. The model will be of the following form:

$$
   Y_{is}  =\beta_i X_i + \gamma_{is} Z_{is} + u_i + \varepsilon_{is}
$$
$$
   \ddot{Y}_{is} = \gamma_{is} \ddot{Z}_{is} + \ddot \eta_{is} 
$$
where $u_i$ is both unobserved and correlated with $Z_{is}$.


The first regression model will be estimated with pooled OLS and the second model will be estimated using both fixed effects and OLS. In other words, I’m going to have you estimate the model using canned routines in Stata and R with individual fixed effects, as well as demean the data manually and estimate the demeaned regression using OLS.

First notice that the second regression has a different notation on the dependent and independent variable; it represents the fact that the variables are columns of demeaned variables. Thus $\ddot{Y}_{is} = Y_{is} - \overline{Y}_i$.
 
Secondly, notice that the time-invariant $X_i$ variables are missing from the second equation. Do you understand why that is the case? These variables have also been demeaned, but since the demeaning is across time, and since these time-invariant variables do not change over time, the demeaning deletes them from the expression. Notice also, that the unobserved individual specific heterogeneity, $u_i$, has disappeared. It has disappeared for the same reason that the $X_i$ terms are gone—because the mean of $u_i$ over time is itself, and thus the demeaning deletes it.


Let’s examine these models:

In [4]:
library(tidyverse)
library(haven)
library(estimatr)
library(plm)

In [7]:
read_data <- function(df)
{
  full_path <- paste("https://github.com/scunning1975/mixtape/raw/master/", 
                     df, sep = "")
  df <- read_dta(full_path)
  return(df)
}

sasp <- read_data("sasp_panel.dta")

In [8]:
#-- Delete all NA
sasp <- na.omit(sasp)

#-- order by id and session 
sasp <- sasp %>% 
  arrange(id, session)

In [11]:
head(sasp, 3)

id,session,age,age_cl,appearance_cl,bmi,schooling,asq_cl,provider_second,asian_cl,⋯,hispanic,other,white,asq,cohab,married,divorced,separated,nevermarried,widowed
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl+lbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,23,46,5,20.48283,14,2116,1,0,⋯,0,1,0,529,0,0,0,0,1,0
1,2,23,33,6,20.48283,14,1089,1,1,⋯,0,1,0,529,0,0,0,0,1,0
1,4,23,61,4,20.48283,14,3721,1,0,⋯,0,1,0,529,0,0,0,0,1,0


In [12]:
#Balance Data
balanced_sasp <- make.pbalanced(sasp, 
                                balance.type = "shared.individuals")

In [13]:
head(balanced_sasp, 3)

id,session,age,age_cl,appearance_cl,bmi,schooling,asq_cl,provider_second,asian_cl,⋯,hispanic,other,white,asq,cohab,married,divorced,separated,nevermarried,widowed
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl+lbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6,1,29,32.5,6,30.89355,16,1056.25,1,0,⋯,0,0,1,841,1,0,0,0,0,0
6,2,29,30.0,8,30.89355,16,900.00,1,0,⋯,0,0,1,841,1,0,0,0,0,0
6,3,29,45.0,4,30.89355,16,2025.00,1,0,⋯,0,0,1,841,1,0,0,0,0,0


In [16]:
balanced_sasp <- balanced_sasp %>% mutate( 
  demean_lnw = lnw - ave(lnw, id),
  demean_age = age - ave(age, id),
  demean_asq = asq - ave(asq, id),
  demean_bmi = bmi - ave(bmi, id),
  demean_hispanic = hispanic - ave(hispanic, id),
  demean_black = black - ave(black, id),
  demean_other = other - ave(other, id),
  demean_asian = asian - ave(asian, id),
  demean_schooling = schooling - ave(schooling, id),
  demean_cohab = cohab - ave(cohab, id),
  demean_married = married - ave(married, id),
  demean_divorced = divorced - ave(divorced, id),
  demean_separated = separated - ave(separated, id),
  demean_age_cl = age_cl - ave(age_cl, id),
  demean_unsafe = unsafe - ave(unsafe, id),
  demean_llength = llength - ave(llength, id),
  demean_reg = reg - ave(reg, id),
  demean_asq_cl = asq_cl - ave(asq_cl, id),
  demean_appearance_cl = appearance_cl - ave(appearance_cl, id),
  demean_provider_second = provider_second - ave(provider_second, id),
  demean_asian_cl = asian_cl - ave(asian_cl, id),
  demean_black_cl = black_cl - ave(black_cl, id),
  demean_hispanic_cl = hispanic_cl - ave(hispanic_cl, id),
  demean_othrace_cl = othrace_cl - ave(othrace_cl, id),
  demean_hot = hot - ave(hot, id),
  demean_massage_cl = massage_cl - ave(massage_cl, id)
  )

In [24]:
#-- POLS
ols <- lm_robust(lnw ~ age + asq + bmi + hispanic + black + other + asian + schooling + cohab + married + divorced + separated + 
           age_cl + unsafe + llength + reg + asq_cl + appearance_cl + provider_second + asian_cl + black_cl + hispanic_cl + 
           othrace_cl + hot + massage_cl, data = balanced_sasp)
summary(ols)


Call:
lm_robust(formula = lnw ~ age + asq + bmi + hispanic + black + 
    other + asian + schooling + cohab + married + divorced + 
    separated + age_cl + unsafe + llength + reg + asq_cl + appearance_cl + 
    provider_second + asian_cl + black_cl + hispanic_cl + othrace_cl + 
    hot + massage_cl, data = balanced_sasp)

Standard error type:  HC2 

Coefficients:
                  Estimate Std. Error  t value  Pr(>|t|)   CI Lower   CI Upper
(Intercept)      7.007e+00  0.3411810  20.5383 1.651e-78  6.3377753  7.6767975
age              2.753e-03  0.0116454   0.2364 8.132e-01 -0.0200995  0.0256049
asq             -1.282e-04  0.0001567  -0.8185 4.133e-01 -0.0004357  0.0001792
bmi             -2.170e-02  0.0022856  -9.4922 1.598e-20 -0.0261809 -0.0172105
hispanic        -2.259e-01  0.0831269  -2.7172 6.697e-03 -0.3889938 -0.0627484
black            2.837e-02  0.0646957   0.4385 6.611e-01 -0.0985874  0.1553218
other           -1.116e-01  0.0770601  -1.4478 1.480e-01 -0.2627867  0.0396486


In [20]:
#-- FE
formula <- as.formula("lnw ~ age + asq + bmi + hispanic + black + other + asian + schooling + 
                      cohab + married + divorced + separated + 
                      age_cl + unsafe + llength + reg + asq_cl + appearance_cl + 
                      provider_second + asian_cl + black_cl + hispanic_cl + 
                      othrace_cl + hot + massage_cl")

model_fe <- lm_robust(formula = formula,
                  data = balanced_sasp, 
                  fixed_effect = ~id, 
                  se_type = "stata")

summary(model_fe)

12 coefficients  not defined because the design matrix is rank deficient





Call:
lm_robust(formula = formula, data = balanced_sasp, fixed_effects = ~id, 
    se_type = "stata")

Standard error type:  HC1 

Coefficients: (12 not defined because the design matrix is rank deficient)
                  Estimate Std. Error   t value  Pr(>|t|)   CI Lower   CI Upper
age                     NA         NA        NA        NA         NA         NA
asq                     NA         NA        NA        NA         NA         NA
bmi                     NA         NA        NA        NA         NA         NA
hispanic                NA         NA        NA        NA         NA         NA
black                   NA         NA        NA        NA         NA         NA
other                   NA         NA        NA        NA         NA         NA
asian                   NA         NA        NA        NA         NA         NA
schooling               NA         NA        NA        NA         NA         NA
cohab                   NA         NA        NA        NA         NA     

In [21]:
#-- Demean OLS
dm_formula <- as.formula("demean_lnw ~ demean_age + demean_asq + demean_bmi + 
                demean_hispanic + demean_black + demean_other +
                demean_asian + demean_schooling + demean_cohab + 
                demean_married + demean_divorced + demean_separated +
                demean_age_cl + demean_unsafe + demean_llength + demean_reg + 
                demean_asq_cl + demean_appearance_cl + 
                demean_provider_second + demean_asian_cl + demean_black_cl + 
                demean_hispanic_cl + demean_othrace_cl +
                demean_hot + demean_massage_cl")

ols_demean <- lm_robust(formula = dm_formula, 
                data = balanced_sasp, clusters = id,
                se_type = "stata")

summary(ols_demean)

12 coefficients  not defined because the design matrix is rank deficient





Call:
lm_robust(formula = dm_formula, data = balanced_sasp, clusters = id, 
    se_type = "stata")

Standard error type:  stata 

Coefficients: (12 not defined because the design matrix is rank deficient)
                         Estimate Std. Error  t value  Pr(>|t|)   CI Lower
(Intercept)             1.822e-18  1.202e-18   1.5156 1.309e-01 -5.453e-19
demean_age                     NA         NA       NA        NA         NA
demean_asq                     NA         NA       NA        NA         NA
demean_bmi                     NA         NA       NA        NA         NA
demean_hispanic                NA         NA       NA        NA         NA
demean_black                   NA         NA       NA        NA         NA
demean_other                   NA         NA       NA        NA         NA
demean_asian                   NA         NA       NA        NA         NA
demean_schooling               NA         NA       NA        NA         NA
demean_cohab                   NA         NA